In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
img_height, img_width = 32, 32
batch_size = 32

data_dir = "Data/Train"
class_names = sorted(os.listdir(data_dir), key=lambda x: int(x))

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=class_names
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    class_names=class_names
)


In [ ]:
train_ds.class_names

In [ ]:
#Divides every pixel by 255
normalization = tf.keras.layers.Rescaling(1/255.0)

# Data augmentation block
data_augmentation = tf.keras.Sequential([
    layers.RandomRotation(0.1),            # rotates ±10%
    layers.RandomZoom(0.1),                # zooms in/out
    layers.RandomContrast(0.1)             # adjusts contrast
])

def preprocess_train(image, label):
    image = normalization(image)
    image = data_augmentation(image)
    return image, label

def preprocess_val(image, label):
    image = normalization(image)
    return image, label

#Applies the normalization without affecting the labels
normalized_train_ds = train_ds.map(preprocess_train)
normalized_val_ds = val_ds.map(preprocess_val)

In [ ]:
num_classes = 43  # 43 classes

model = models.Sequential([
    Input(shape=(32,32,3)),

    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(normalized_train_ds,validation_data = normalized_val_ds, epochs=20)

In [ ]:
y_true = []
y_pred = []

# Loop over validation dataset
for images, labels in normalized_val_ds:
    preds = model.predict(images, verbose=0)         # predictions (probabilities)
    pred_classes = np.argmax(preds, axis=1)          # convert to class IDs
    y_pred.extend(pred_classes)
    y_true.extend(labels.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# Compute accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Validation Accuracy (from predictions): {acc:.4f}")


In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10,8))  # bigger figure
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title("Confusion Matrix", fontsize=16)
plt.colorbar()

# Add class labels
num_classes = cm.shape[0]
tick_marks = np.arange(num_classes)
plt.xticks(tick_marks, tick_marks, fontsize=8, rotation=90)
plt.yticks(tick_marks, tick_marks, fontsize=8)

# Add numbers in cells
thresh = cm.max() / 2.0
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, format(cm[i, j], 'd'),
                 ha="center", va="center", fontsize=6,
                 color="white" if cm[i, j] > thresh else "black")

plt.ylabel("True Label", fontsize=14)
plt.xlabel("Predicted Label", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array
from PIL import UnidentifiedImageError

test_path = "Data/Test"

valid_exts = (".png", ".jpg", ".jpeg", ".bmp")  # accepted image formats

imgs = []
names = []

for fname in sorted(os.listdir(test_path)):
    if not fname.lower().endswith(valid_exts):
        continue  # skip non-images
    try:
        img = load_img(os.path.join(test_path, fname), target_size=(32, 32))
        img_array = img_to_array(img) / 255.0
        imgs.append(img_array)
        names.append(fname)
    except UnidentifiedImageError:
        print(f"⚠️ Skipping corrupted file: {fname}")
        continue

imgs = np.array(imgs)

# Predict all in one go
preds = model.predict(imgs, batch_size=128, verbose=1)
pred_classes = np.argmax(preds, axis=1)

# Plot in batches of 100 (10x10 grid)
batch_size = 100
for start in range(0, len(imgs), batch_size):
    end = start + batch_size
    batch_imgs = imgs[start:end]
    batch_preds = pred_classes[start:end]
    batch_names = names[start:end]

    fig, axes = plt.subplots(10, 10, figsize=(15, 15))
    axes = axes.ravel()

    for i in range(len(batch_imgs)):
        axes[i].imshow(batch_imgs[i])
        axes[i].set_title(f"{batch_names[i]}\nPred: {batch_preds[i]}", fontsize=10)
        axes[i].axis("off")

    for j in range(len(batch_imgs), 100):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()
